### Problema

In [1]:
%%html
<img src="LoL Map.jpg",width=100,height=100>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

In [2]:
data = pd.read_csv('games.csv')
data.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 61 columns):
gameId                51490 non-null int64
creationTime          51490 non-null int64
gameDuration          51490 non-null int64
seasonId              51490 non-null int64
winner                51490 non-null int64
firstBlood            51490 non-null int64
firstTower            51490 non-null int64
firstInhibitor        51490 non-null int64
firstBaron            51490 non-null int64
firstDragon           51490 non-null int64
firstRiftHerald       51490 non-null int64
t1_champ1id           51490 non-null int64
t1_champ1_sum1        51490 non-null int64
t1_champ1_sum2        51490 non-null int64
t1_champ2id           51490 non-null int64
t1_champ2_sum1        51490 non-null int64
t1_champ2_sum2        51490 non-null int64
t1_champ3id           51490 non-null int64
t1_champ3_sum1        51490 non-null int64
t1_champ3_sum2        51490 non-null int64
t1_champ4id           51490 non

### Dividing the data

In [4]:
X = data.drop(['gameId','winner'], axis=1)
y = data.winner

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

### Data Standardization

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [9]:
X_train_scaler = pd.DataFrame(X_train_scaler, index=X_train.index, columns=X_train.columns)
X_test_scaler = pd.DataFrame(X_test, index=X_test.index, columns=X_test.columns)

In [10]:
X_train_scaler.head()

,creationTime,gameDuration,seasonId,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
43033,0.266242,-0.270637,0.0,-0.906913,1.012679,-0.458045,0.086618,-0.776890,-0.889479,0.162581,...,-1.178611,-0.785435,-0.675396,-1.144635,-0.563407,0.320187,-0.533005,0.168998,-0.667123,3.122364
46451,-0.147065,-0.975007,0.0,1.015761,1.012679,-1.938801,-1.102704,0.981345,-0.889479,-0.860742,...,-0.143052,-0.785435,-0.675396,0.487552,-0.563407,-0.401064,-0.231095,-0.542363,-0.667123,-0.322750
39914,0.565565,0.359382,0.0,1.015761,1.012679,-0.458045,-1.102704,-0.776890,-0.889479,-0.927845,...,-0.660832,-0.785435,-0.675396,-1.144635,-0.563407,0.476133,0.109770,-0.483895,1.249317,3.969044
41280,0.702933,-0.622822,0.0,1.015761,1.012679,1.022711,1.275940,-0.776890,1.539626,-0.105832,...,1.410287,0.807124,0.951939,-0.328541,1.774916,-0.664223,-0.494048,-0.737256,-0.541296,0.446075
51144,0.680969,-0.188461,0.0,1.015761,1.012679,1.022711,1.275940,0.981345,1.539626,-0.651044,...,0.633618,0.010845,0.951939,-0.328541,1.774916,-0.878649,1.268713,-0.094108,-0.754234,-0.887204


### Feature Selection
Often you'll have hundreds or thousands of features after various encodings and feature generation. This can lead to two problems. First, the more features you have, the more likely you are to overfit to the training and validation sets. This will cause your model to perform worse at generalizing to new data.

Secondly, the more features you have, the longer it will take to train your model and optimize hyperparameters. Also, when building user-facing products, you'll want to make inference as fast as possible. Using fewer features can speed up inference at the cost of predictive performance.

To help with these issues, you'll want to use feature selection techniques to keep the most informative features for your model.

#### Univariate Feature Selection
The simplest and fastest methods are based on univariate statistical tests. For each feature, measure how strongly the target depends on the feature using a statistical test like  χ2  or ANOVA.

From the scikit-learn feature selection module, feature_selection.SelectKBest returns the K best features given some scoring function. For our classification problem, the module provides three different scoring functions:  χ2 , ANOVA F-value, and the mutual information score. The F-value measures the linear dependency between the feature variable and the target. This means the score might underestimate the relation between a feature and the target if the relationship is nonlinear. The mutual information score is nonparametric and so can capture nonlinear relationships.

With SelectKBest, we define the number of features to keep, based on the score from the scoring function. Using .fit_transform(features, target) we get back an array with only the selected features.

In [11]:
from sklearn.feature_selection import SelectKBest, f_classif

In [12]:
parameters = list()
results = list()
for n_estimators in [10, 30,50,100,150,200]:
    for max_depth in [3,5,7,10,16,20,25,30,40,50]:
        for n_features in [5, 10, 16, 20, 26, 30, 40, 50]:
            #Getting the columns
            feature_cols = X_train.columns

            #
            selector = SelectKBest(f_classif, k=n_features)
            X_skb = selector.fit_transform(X_train[feature_cols],y_train)
            #
            selected_features = pd.DataFrame(selector.inverse_transform(X_skb),
                                             index=X_train.index,
                                             columns=feature_cols)
        
        
            # Dropped columns have values of all 0s, so var is 0, drop them
            selected_columns = selected_features.columns[selected_features.var() != 0]

            X_train_Univariate = selected_features[selected_columns]
            X_test_Univariate = X_test[selected_columns]
            
            rfc = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
            score2 = cross_validate(rfc,X_train_Univariate,y_train,scoring=['accuracy','precision','recall'],cv=3)
            parameters.append([n_estimators,max_depth,n_features])
            result = (score2["test_accuracy"].sum()+score2["test_precision"].sum()+score2["test_recall"].sum())/9
            results.append(result)
            print("Parameters = ", [n_estimators,max_depth,n_features])
            print("Result = ",result)

/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Parameters =  [10, 3, 5]
Result =  0.9554349788320917


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Parameters =  [10, 3, 10]
Result =  0.9580809936606889


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


...

Parameters =  [200, 50, 30]
Result =  0.9700397041562923


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Parameters =  [200, 50, 40]
Result =  0.9694157504620561


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Parameters =  [200, 50, 50]
Result =  0.9699850171261579


In [13]:
#Best parameters
print(parameters[results.index(max(results))])

[200, 16, 20]


In [15]:
from sklearn.metrics import classification_report

In [16]:
#
selector = SelectKBest(f_classif, k=20)
X_skb = selector.fit_transform(X_train[feature_cols],y_train)
#
selected_features = pd.DataFrame(selector.inverse_transform(X_skb),
                                 index=X_train.index,
                                 columns=feature_cols)
        
        
# Dropped columns have values of all 0s, so var is 0, drop them
selected_columns = selected_features.columns[selected_features.var() != 0]

X_train_Univariate = selected_features[selected_columns]
X_test_Univariate = X_test[selected_columns]


rfc1 = RandomForestClassifier(n_estimators=200, max_depth=16)
rfc1.fit(X_train_Univariate,y_train)
y_pred = rfc1.predict(X_test_Univariate)
score = classification_report(y_test, y_pred)
print(score)

/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [2] are constant.
  UserWarning)
/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


              precision    recall  f1-score   support

           1       0.97      0.97      0.97      5215
           2       0.97      0.97      0.97      5083

    accuracy                           0.97     10298
   macro avg       0.97      0.97      0.97     10298
weighted avg       0.97      0.97      0.97     10298



In [17]:
X_train_Univariate.head()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ5id,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_champ2id,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
43033,1695,1,2,1,1,1,0,134,57,10,2,1,2,0,43,1,0,0,0,0
46451,1335,2,2,0,0,2,0,12,127,0,0,0,0,0,89,5,0,0,2,0
39914,2017,2,2,1,0,1,0,4,75,9,2,0,3,0,89,3,0,0,0,0
41280,1515,2,2,2,2,1,2,102,420,1,0,0,1,0,119,11,2,1,1,1
51144,1737,2,2,2,2,2,2,37,32,4,0,0,1,0,222,8,1,1,1,1


#### L1 regularization
Univariate methods consider only one feature at a time when making a selection decision. Instead, we can make our selection using all of the features by including them in a linear model with L1 regularization. This type of regularization (sometimes called Lasso) penalizes the absolute magnitude of the coefficients, as compared to L2 (Ridge) regression which penalizes the square of the coefficients.

As the strength of regularization is increased, features which are less important for predicting the target are set to 0. This allows us to perform feature selection by adjusting the regularization parameter. We choose the parameter by finding the best performance on a hold-out set, or decide ahead of time how many features to keep.

For regression problems you can use sklearn.linear_model.Lasso, or sklearn.linear_model.LogisticRegression for classification. These can be used along with sklearn.feature_selection.SelectFromModel to select the non-zero coefficients. Otherwise, the code is similar to the univariate tests.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [12]:
parameters = list()
results = list()
for n_estimators in [10, 30,50,100,150,200]:
    for max_depth in [3,5,7,10,16,20,25,30,40,50]:
        for n_features in [5, 10, 16, 20, 26, 30, 40, 50]:
            #Getting the columns
            feature_cols = X_train.columns

            # Set the regularization parameter C=1
            logistic = LogisticRegression(C=1, penalty="l1", random_state=42).fit(X_train_scaler, y_train)
            model = SelectFromModel(logistic, prefit=True, threshold=-np.inf,max_features=n_features)

            X_logistic = model.transform(X_train)
        
        
            # Get back the kept features as a DataFrame with dropped columns as all 0s
            selected_features = pd.DataFrame(model.inverse_transform(X_logistic), 
                                             index=X_train.index,
                                             columns=X_train.columns)

            # Dropped columns have values of all 0s, keep other columns 
            selected_columns = selected_features.columns[selected_features.var() != 0]
            
            X_train_L1 = X_train[selected_columns]
            X_test_L1 = X_test[selected_columns]
            
            X_train_L1.head()
            
            rfc = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
            score2 = cross_validate(rfc,X_train_L1,y_train,scoring=['accuracy','precision','recall'],cv=3)
            parameters.append([n_estimators,max_depth,n_features])
            result = (score2["test_accuracy"].sum()+score2["test_precision"].sum()+score2["test_recall"].sum())/9
            results.append(result)
            print("Parameters = ", [n_estimators,max_depth,n_features])
            print("Result = ",result)

/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Parameters =  [10, 3, 5]
Result =  0.952668750791519


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Parameters =  [10, 3, 10]
Result =  0.9457813227172781


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

Parameters =  [200, 50, 30]
Result =  0.9699819090694469


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Parameters =  [200, 50, 40]
Result =  0.9701301269746053


/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Parameters =  [200, 50, 50]
Result =  0.9699006955741696


In [13]:
#Best parameters
print(parameters[results.index(max(results))])

[200, 16, 26]


In [16]:
#Getting the columns
feature_cols = X_train.columns

# Set the regularization parameter C=1
logistic = LogisticRegression(C=1, penalty="l1", random_state=42).fit(X_train_scaler, y_train)
model = SelectFromModel(logistic, prefit=True, threshold=-np.inf,max_features=26)
X_logistic = model.transform(X_train)
        
# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(model.inverse_transform(X_logistic), 
                                 index=X_train.index,
                                 columns=X_train.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]
            
X_train_L1 = X_train[selected_columns]
X_test_L1 = X_test[selected_columns]


rfc = RandomForestClassifier(n_estimators=200, max_depth=16)
rfc.fit(X_train_L1, y_train)
y_pred=rfc.predict(X_test_L1)
score = classification_report(y_test, y_pred)
print(score)

/home/chsevero/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           1       0.97      0.97      0.97      5215
           2       0.97      0.97      0.97      5083

    accuracy                           0.97     10298
   macro avg       0.97      0.97      0.97     10298
weighted avg       0.97      0.97      0.97     10298



In [17]:
selected_columns

Index(['gameDuration', 'firstBlood', 'firstTower', 'firstInhibitor',
       'firstDragon', 'firstRiftHerald', 't1_champ1_sum2', 't1_champ3id',
       't1_champ3_sum1', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't2_champ1_sum2',
       't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3_sum1', 't2_champ3_sum2',
       't2_champ5_sum1', 't2_champ5_sum2', 't2_towerKills',
       't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHeraldKills'],
      dtype='object')